In [31]:
import tensorflow as tf
import numpy as np
import os
import urllib
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import SparseCategoricalCrossentropy


import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler




In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading the database of office scripts which will be used in order to generate texts, and trasnforming it into a string file

In [34]:

target_url='https://raw.githubusercontent.com/tsaruggan/dunder-mifflin-RNNfinity/master/the-office.txt'

text=""
for line in urllib.request.urlopen(target_url):
  if (line.decode(encoding='utf-8').startswith('\n'))==False:
    text+=line.decode(encoding='utf-8')
  

Checking that the text file was created correctly 

In [35]:

   
#print(lst_of_characters[:30])  
lst_of_characters=text.splitlines()
print(lst_of_characters[:30])


['MICHAEL:', 'All right Jim. Your quarterlies look very good. How are things at the library?', 'JIM:', "Oh, I told you. I couldn't close it. So...", 'MICHAEL:', "So you've come to the master for guidance? Is this what you're saying, grasshopper?", 'JIM:', 'Actually, you called me in here, but yeah.', 'MICHAEL:', "All right. Well, let me show you how it's done.", 'MICHAEL:', "[on the phone] Yes, I'd like to speak to your office manager, please. Yes, hello. This is Michael Scott. I am the Regional Manager of Dunder Mifflin Paper Products. Just wanted to talk to you manager-a-manger. [quick cut scene] All right. Done deal. Thank you very much, sir. You're a gentleman and a scholar. Oh, I'm sorry. OK. I'm sorry. My mistake. [hangs up] That was a woman I was talking to, so... She had a very low voice. Probably a smoker, so... [Clears throat] So that's the way it's done.", 'MICHAEL:', "I've, uh, I've been at Dunder Mifflin for 12 years, the last four as Regional Manager. If you want to come 

We wanted to create a dialogue between Dwight and Jim, and so we split the text file into three seperate files: one containing only lines spoken by Jim, one containing only lines spoken by Dwight, and one containing every instance in which the two charcters spoke to each other. 

We would then try to generate our own netwrok by utilizing two different methods:

1. Basing the model on the "Dialogue" text, in an attempt to get the network to learn the nature of conversations by Jim and Dwight, and then generate them.

2. Building two diffentent model: a "Jim network" which will be based on the Jim lines, and a "Dwight network" which will be based on the Dwight dialogues. after building these networks, we will attempt to make the models "speak" to each other in an attempt to create a text similar to a real conversation between Jim and Dwight

In [36]:
#lst_of_characters=text.splitlines()
#c_lst=lst_of_characters.copy()
dialogue_lst=[]
ind_error=[len(lst_of_characters),len(lst_of_characters)-1,len(lst_of_characters)-2]
for ind in range(len(lst_of_characters)):
  if (ind not in ind_error) and (ind%2==0) and (lst_of_characters[ind]=='JIM:') and (lst_of_characters[ind+2]=='DWIGHT:') :
        dialogue_lst.append(lst_of_characters[ind])
        dialogue_lst.append(lst_of_characters[ind+1])
        dialogue_lst.append(lst_of_characters[ind+2])
        dialogue_lst.append(lst_of_characters[ind+3])
  if (ind not in ind_error) and (ind!=0) and (ind%2==0) and (lst_of_characters[ind]=='DWIGHT:') and (lst_of_characters[ind+2]=='JIM:') and lst_of_characters[ind-2]!='JIM:' :
        dialogue_lst.append(lst_of_characters[ind])
        dialogue_lst.append(lst_of_characters[ind+1])
        dialogue_lst.append(lst_of_characters[ind+2])
        dialogue_lst.append(lst_of_characters[ind+3])

jim_text = []
ind_error=[len(lst_of_characters),len(lst_of_characters)-1,len(lst_of_characters)-2]
for ind in range(len(lst_of_characters)):
    if (ind not in ind_error) and (ind%2==0) and (lst_of_characters[ind]=='JIM:'):
      jim_text.append(lst_of_characters[ind])
      jim_text.append(lst_of_characters[ind+1])


dwight_text = []
ind_error=[len(lst_of_characters),len(lst_of_characters)-1,len(lst_of_characters)-2]
for ind in range(len(lst_of_characters)):
    if (ind not in ind_error) and (ind%2==0) and (lst_of_characters[ind]=='DWIGHT:'):
      dwight_text.append(lst_of_characters[ind])
      dwight_text.append(lst_of_characters[ind+1])

      

Checking that the text files were created correctly 

In [37]:
len(dialogue_lst), len(jim_text), len(dwight_text)
jim_text[:1000]

['JIM:',
 "Oh, I told you. I couldn't close it. So...",
 'JIM:',
 'Actually, you called me in here, but yeah.',
 'JIM:',
 "My job is to speak to clients on the phone about... uh, quantities and type of copier paper. You know, whether we can supply it to them. Whether they can pay for it. And... I'm boring myself just talking about this.",
 'JIM:',
 'Whassup! I still love that after seven years.',
 'JIM:',
 'Nothing.',
 'JIM:',
 'All right. Take care.',
 'JIM:',
 'Yes?',
 'JIM:',
 'Yeah, stop. That is ridiculous.',
 'JIM:',
 'Sure. Mr. Davis, let me call you right back. Yeah, something just came up. Two minutes. Thank you very much. Dwight, what are you doing?',
 'JIM:',
 'What are you doing?',
 'JIM:',
 "It's not on your desk.",
 'JIM:',
 'Why not?',
 'JIM:',
 "[crosses fingers] We'll see. [Dwight begins smashing pencils with his phone] This is why the whole downsizing thing just doesn't bother me.",
 'JIM:',
 "If I left, what would I do with all this useless information in my head? Yo

In [38]:
'''
def new_text(text):
  new_text = []
  check = 0 
  for line in text:
    if line[:4] == 'JIM:':
      check = 1 
      continue
    if line[:4] == 'DWIG':
       check = 2
       continue
    if check == 1:
      new_text.append('Jim: ' + line)
    if check == 2:
      new_text.append('Dwight: ' + line)
  
  return new_text
'''

"\ndef new_text(text):\n  new_text = []\n  check = 0 \n  for line in text:\n    if line[:4] == 'JIM:':\n      check = 1 \n      continue\n    if line[:4] == 'DWIG':\n       check = 2\n       continue\n    if check == 1:\n      new_text.append('Jim: ' + line)\n    if check == 2:\n      new_text.append('Dwight: ' + line)\n  \n  return new_text\n"

In [39]:
'''
jim_text= new_text(jim_text)
dwight_text= new_text(dwight_text)
dialogue_lst =  new_text(dialogue_lst)
'''


'\njim_text= new_text(jim_text)\ndwight_text= new_text(dwight_text)\ndialogue_lst =  new_text(dialogue_lst)\n'

checking that the text files were created correctly:

In [40]:
jim_text[:1000]

['JIM:',
 "Oh, I told you. I couldn't close it. So...",
 'JIM:',
 'Actually, you called me in here, but yeah.',
 'JIM:',
 "My job is to speak to clients on the phone about... uh, quantities and type of copier paper. You know, whether we can supply it to them. Whether they can pay for it. And... I'm boring myself just talking about this.",
 'JIM:',
 'Whassup! I still love that after seven years.',
 'JIM:',
 'Nothing.',
 'JIM:',
 'All right. Take care.',
 'JIM:',
 'Yes?',
 'JIM:',
 'Yeah, stop. That is ridiculous.',
 'JIM:',
 'Sure. Mr. Davis, let me call you right back. Yeah, something just came up. Two minutes. Thank you very much. Dwight, what are you doing?',
 'JIM:',
 'What are you doing?',
 'JIM:',
 "It's not on your desk.",
 'JIM:',
 'Why not?',
 'JIM:',
 "[crosses fingers] We'll see. [Dwight begins smashing pencils with his phone] This is why the whole downsizing thing just doesn't bother me.",
 'JIM:',
 "If I left, what would I do with all this useless information in my head? Yo

In [41]:
dwight_text[:1000]

['DWIGHT:',
 '[singing] Shall I play for you? Pa rum pump um pum [Imitates heavy drumming] I have no gifts for you. Pa rum pump um pum [Imitates heavy drumming]',
 'DWIGHT:',
 'Whassup!',
 'DWIGHT:',
 'Whassup.',
 'DWIGHT:',
 'What?',
 'DWIGHT:',
 "Just clearing my desk. I can't concentrate.",
 'DWIGHT:',
 "It's overlapping. It's all spilling over the edge. One word, two syllables. Demarcation.",
 'DWIGHT:',
 "You can't do that.",
 'DWIGHT:',
 'Safety violation. I could fall and pierce an organ.',
 'DWIGHT:',
 'Downsizing?',
 'DWIGHT:',
 'Downsizing? I have no problem with that. I have been recommending downsizing since I first got here. I even brought it up in my interview. I say, bring it on.',
 'DWIGHT:',
 "I'm Assistant Regional Manager. I should know first.",
 'DWIGHT:',
 'OK, um, can you just tell me please? Just tell me quietly. Can you whisper it in my ear?',
 'DWIGHT:',
 "Please, OK? Do you want me to tell 'em?",
 'DWIGHT:',
 "OK. You tell 'em. With my permission. Permission g

In [42]:
dialogue_lst[:1000]

['DWIGHT:',
 '[singing] Shall I play for you? Pa rum pump um pum [Imitates heavy drumming] I have no gifts for you. Pa rum pump um pum [Imitates heavy drumming]',
 'JIM:',
 "My job is to speak to clients on the phone about... uh, quantities and type of copier paper. You know, whether we can supply it to them. Whether they can pay for it. And... I'm boring myself just talking about this.",
 'JIM:',
 'Sure. Mr. Davis, let me call you right back. Yeah, something just came up. Two minutes. Thank you very much. Dwight, what are you doing?',
 'DWIGHT:',
 'What?',
 'JIM:',
 'What are you doing?',
 'DWIGHT:',
 "Just clearing my desk. I can't concentrate.",
 'JIM:',
 "It's not on your desk.",
 'DWIGHT:',
 "It's overlapping. It's all spilling over the edge. One word, two syllables. Demarcation.",
 'DWIGHT:',
 "You can't do that.",
 'JIM:',
 'Why not?',
 'JIM:',
 'Why not?',
 'DWIGHT:',
 'Safety violation. I could fall and pierce an organ.',
 'JIM:',
 "[crosses fingers] We'll see. [Dwight begins 

We will now generate the functions used in tokenization, vocabulary creation, model creation, train-val split etc. 

We will implement these functions when generating our three models 

In [43]:
def create_text(text, name=None):
  text="\n".join(text)
  text = text.replace("\n", " ")
  if name == 'JIM' or name == 'DWIGHT':
    text= text.replace(name+':', "")
    text = name + ":" + text
    
  return text


def create_sequences(text, ids_dataset):
  seq_length = 100
  examples_per_epoch = len(text)//(seq_length+1)
  sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
  return sequences

def create_vocabulary(sequence):
  vocab = sorted(set(dialogue_text))
  ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
  chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
  all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
  ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

  return ids_from_chars, chars_from_ids, all_ids, ids_dataset, vocab


def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

def create_buffer(dataset):
  # Batch size
  BATCH_SIZE = 64

  # Buffer size to shuffle the dataset
  # (TF data is designed to work with possibly infinite sequences,
  # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
  # it maintains a buffer in which it shuffles elements).
  BUFFER_SIZE = 10000

  Dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
  
  return Dataset


def create_model(dataset):
  office_gru_model = GRUModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

  for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = office_gru_model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

    return office_gru_model


def create_checkpoint(model_name, epochs):
  checkpoint_dir = os.path.join('/content/drive/My Drive/nlp_checkpoints', model_name)
  checkpoint_prefix = os.path.join(checkpoint_dir, str(epochs) + "_epochs", str(epochs) + "_epochs_cp.ckpt")

  checkpoint_callback = ModelCheckpoint(
      filepath=checkpoint_prefix,
      save_weights_only=True,
      save_best_only=True,
      monitor='val_loss',
      mode='min',
      verbose=1)
  return checkpoint_callback, checkpoint_prefix


def create_train_val(dataset):
  train_size = int(0.8 * len(dataset))
  val_size = int(0.2 * len(dataset))
  
  train_dataset = dataset.take(train_size)
  val_dataset = dataset.skip(train_size)

  return train_dataset, val_dataset



Using the text generation function to remove unnecessary  characters

In [44]:
jim_text = create_text(jim_text, 'JIM')
dwight_text = create_text(dwight_text, 'DWIGHT')
dialogue_text = create_text(dialogue_lst)

Checking the generated text

In [45]:
dialogue_text[1000:2000]


": Downsizing? DWIGHT: OK you know what, you can be a witness. [points to Ryan] Can you reprimand him? JIM: How do you know it was me? JIM: How do you know it was me? DWIGHT: It's always you. Are you going to discipline him or not? JIM: Yeah, definitely. You too. Enjoy it. [looks at camera] You know what, just come here. DWIGHT: Dwight Schrute. My father's name, also Dwight Schrute. My grandfather's name, Dweide Schrude, Amish. That's my family. I don't know where they came, the Amish, came from originally. Uh, Amland. JIM: Do you really have to do that right now? DWIGHT: Yes I do. I should have done it weeks ago actually. JIM: Mr. Decker, I'm sorry about that. What were you... Can you hold on one second? Yeah, just one second. Thanks. [Power off, silence] Hello? That's it. Perfect. So what I was saying... [Dialing tone] Hello? Thanks, Dwight. DWIGHT: Retaliation. Tit for tit. JIM: That is not the expression. DWIGHT: Well, it should be. DWIGHT: What you doing? JIM: Freecell. JIM: Freec

Generating vocabulary and creating sequences

In [46]:
ids_from_chars, chars_from_ids, all_ids, ids_dataset, vocab = create_vocabulary(dialogue_text)

jim_sequences = create_sequences(jim_text, ids_dataset)
dwight_sequences = create_sequences(dwight_text, ids_dataset)
dialogue_sequences = create_sequences(dialogue_text, ids_dataset)




Mapping the sequneces

In [47]:
dataset_dialogue = dialogue_sequences.map(split_input_target)
dataset_jim = jim_sequences.map(split_input_target)
dataset_dwight = dwight_sequences.map(split_input_target)


In [48]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [49]:
for input_example, target_example in dataset_dialogue.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'MICHAEL:[UNK]All right Jim. Your quarterlies look very good. How are things at the library?[UNK]JIM:[UNK]Oh, I t'
Target: b'ICHAEL:[UNK]All right Jim. Your quarterlies look very good. How are things at the library?[UNK]JIM:[UNK]Oh, I to'


In [50]:
for input_example, target_example in dataset_jim.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'MICHAEL:[UNK]All right Jim. Your quarterlies look very good. How are things at the library?[UNK]JIM:[UNK]Oh, I t'
Target: b'ICHAEL:[UNK]All right Jim. Your quarterlies look very good. How are things at the library?[UNK]JIM:[UNK]Oh, I to'


In [51]:
#Creating the buffer and getting the vocabulary size

dataset_dialogue = create_buffer(dataset_dialogue)
dataset_jim = create_buffer(dataset_jim)
dataset_dwight = create_buffer(dataset_dwight)


# Length of the vocabulary in chars
vocab_size = len(vocab)
vocab_size

78

In [52]:
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [53]:
class GRUModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

Creating the three models: the dialogue model, the "Jim model" and the the "Dwight model".

In [54]:
office_gru_model_dialogue = create_model(dataset_dialogue)
office_gru_model_dialogue.summary()


(64, 100, 79) # (batch_size, sequence_length, vocab_size)
Model: "gru_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  20224     
                                                                 
 gru_3 (GRU)                 multiple                  3938304   
                                                                 
 dense_3 (Dense)             multiple                  80975     
                                                                 
Total params: 4,039,503
Trainable params: 4,039,503
Non-trainable params: 0
_________________________________________________________________


In [55]:

office_gru_model_jim = create_model(dataset_jim) 
office_gru_model_jim.summary()


(64, 100, 79) # (batch_size, sequence_length, vocab_size)
Model: "gru_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  20224     
                                                                 
 gru_4 (GRU)                 multiple                  3938304   
                                                                 
 dense_4 (Dense)             multiple                  80975     
                                                                 
Total params: 4,039,503
Trainable params: 4,039,503
Non-trainable params: 0
_________________________________________________________________


In [56]:
office_gru_model_dwight = create_model(dataset_dwight) 
office_gru_model_dwight.summary()


(64, 100, 79) # (batch_size, sequence_length, vocab_size)
Model: "gru_model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     multiple                  20224     
                                                                 
 gru_5 (GRU)                 multiple                  3938304   
                                                                 
 dense_5 (Dense)             multiple                  80975     
                                                                 
Total params: 4,039,503
Trainable params: 4,039,503
Non-trainable params: 0
_________________________________________________________________


Creating the loss function and setting epochs

In [57]:
sparse_nll = SparseCategoricalCrossentropy(from_logits=True)
office_gru_model_dialogue.compile(optimizer='adam', loss=sparse_nll)
office_gru_model_jim.compile(optimizer='adam', loss=sparse_nll)
office_gru_model_dwight.compile(optimizer='adam', loss=sparse_nll)

EPOCHS = 20
len(dataset_dialogue)

631

Creating the val-train split

In [58]:
train_dataset_dialogue,val_dataset_dialogue = create_train_val(dataset_dialogue)
train_dataset_jim,val_dataset_jim = create_train_val(dataset_jim)
train_dataset_dwight,val_dataset_dwight = create_train_val(dataset_dwight)


In [59]:
checkpoint_callback_dialogue, checkpoint_prefix_dialogue = create_checkpoint('office_gru_Dialogue', EPOCHS)
checkpoint_callback_jim, checkpoint_prefix_jim = create_checkpoint('office_gru_Jim', EPOCHS)
checkpoint_callback_dwight, checkpoint_prefix_dwight = create_checkpoint('office_gru_Dwight', EPOCHS)


Training the three models we have created

In [60]:
if os.path.exists(os.path.dirname(checkpoint_prefix_dialogue)):
  office_gru_model_dialogue.load_weights(checkpoint_prefix_dialogue)
else:
  office_gru_model_dialogue.fit(train_dataset_dialogue, 
                              validation_data=val_dataset_dialogue, 
                              epochs=EPOCHS,
                              callbacks=[checkpoint_callback_dialogue])

Epoch 1/20
504/504 [==============================] - ETA: 0s - loss: 2.0884
Epoch 1: val_loss improved from inf to 1.55298, saving model to /content/drive/My Drive/nlp_checkpoints/office_gru_Dialogue/20_epochs/20_epochs_cp.ckpt
504/504 [==============================] - 39s 67ms/step - loss: 2.0884 - val_loss: 1.5530
Epoch 2/20
504/504 [==============================] - ETA: 0s - loss: 1.3960
Epoch 2: val_loss improved from 1.55298 to 1.32263, saving model to /content/drive/My Drive/nlp_checkpoints/office_gru_Dialogue/20_epochs/20_epochs_cp.ckpt
504/504 [==============================] - 36s 70ms/step - loss: 1.3960 - val_loss: 1.3226
Epoch 3/20
504/504 [==============================] - ETA: 0s - loss: 1.2620
Epoch 3: val_loss improved from 1.32263 to 1.23515, saving model to /content/drive/My Drive/nlp_checkpoints/office_gru_Dialogue/20_epochs/20_epochs_cp.ckpt
504/504 [==============================] - 38s 73ms/step - loss: 1.2620 - val_loss: 1.2351
Epoch 4/20
504/504 [============

In [61]:
if os.path.exists(os.path.dirname(checkpoint_prefix_jim)):
  office_gru_model_jim.load_weights(checkpoint_prefix_jim)
else:
  office_gru_model_jim.fit(train_dataset_jim, 
                              validation_data=val_dataset_jim, 
                              epochs=EPOCHS,
                              callbacks=[checkpoint_callback_jim])

Epoch 1/20
504/504 [==============================] - ETA: 0s - loss: 2.1061
Epoch 1: val_loss improved from inf to 1.55710, saving model to /content/drive/My Drive/nlp_checkpoints/office_gru_Jim/20_epochs/20_epochs_cp.ckpt
504/504 [==============================] - 38s 72ms/step - loss: 2.1061 - val_loss: 1.5571
Epoch 2/20
504/504 [==============================] - ETA: 0s - loss: 1.4009
Epoch 2: val_loss improved from 1.55710 to 1.32529, saving model to /content/drive/My Drive/nlp_checkpoints/office_gru_Jim/20_epochs/20_epochs_cp.ckpt
504/504 [==============================] - 37s 71ms/step - loss: 1.4009 - val_loss: 1.3253
Epoch 3/20
504/504 [==============================] - ETA: 0s - loss: 1.2641
Epoch 3: val_loss improved from 1.32529 to 1.23819, saving model to /content/drive/My Drive/nlp_checkpoints/office_gru_Jim/20_epochs/20_epochs_cp.ckpt
504/504 [==============================] - 37s 71ms/step - loss: 1.2641 - val_loss: 1.2382
Epoch 4/20
504/504 [===========================

In [ ]:
checkpoint_callback_dwight, checkpoint_prefix_dwight = create_checkpoint('office_gru_Dwight', EPOCHS)

if os.path.exists(os.path.dirname(checkpoint_prefix_dwight)):
  office_gru_model_dwight.load_weights(checkpoint_prefix_dwight)
else:
  office_gru_model_dwight.fit(train_dataset_dwight, 
                              validation_data=val_dataset_dwight, 
                              epochs=EPOCHS,
                              callbacks=[checkpoint_callback_dwight])

In [63]:

# Taken directly from TF guide

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model_dialogue = OneStep(office_gru_model_dialogue, chars_from_ids, ids_from_chars)
one_step_model_jim = OneStep(office_gru_model_jim, chars_from_ids, ids_from_chars)
one_step_model_dwight = OneStep(office_gru_model_dwight, chars_from_ids, ids_from_chars)


In [65]:
from tqdm.notebook import tqdm as notebook_tqdm


def predict_multiple_steps(start_string, one_step_model):
  states = None
  next_char = tf.constant([start_string])
  result = [next_char]

  for n in notebook_tqdm(range(1000)):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

  result = tf.strings.join(result)
  return result

Creating a dialogue between the "Jim" model and the "Dwight" model

In [67]:


n = 0
result_jim = predict_multiple_steps('JIM: What are you doing?', one_step_model_jim)
result_jim = result_jim[0].numpy().decode('utf-8')
combo_result = []
combo_result.append(result_jim)


while n <10:
  result_dwight = predict_multiple_steps(result_jim, one_step_model_dwight)
  result_dwight = result_dwight[0].numpy().decode('utf-8')
  result_jim = predict_multiple_steps(result_dwight, one_step_model_jim)
  result_jim = result_jim[0].numpy().decode('utf-8')
  combo_result.append("\nDwight: ")
  combo_result.append(result_dwight)
  combo_result.append("\nJim: ")
  combo_result.append(result_jim)
  n += 1

combo_result = tf.strings.join(combo_result)







  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [72]:
combo_result

<tf.Tensor: shape=(), dtype=string, numpy=b"JIM: What are you doing?!?! It's not britically that back if he'd chock into the Afternoon dumb. And, everyone here for our own people burger.  And Robert from Andy in basison't. The would you stay for me? And, you should take some conversations. I'm right, Dwight, 'Dogy's ' only the bull the wheels of I Management of I.T.ach early, who knows what I bring and I like to have to do.  I'm trying to make sure Darryl's department, because if nothing but I think of you go ail.  But I agreed to jair to the office and alter of a room at all.  It's that with Babradamia. You know what? I, I miss just what I'm going to do closures. Is she how I bring it? I meant... I'd be indicate that, thanks my returnant rollings.[The best person through the beach and indecer, bad question. So maybe thing. Can we be easier. [Jim places away, he'll breaks' at Gabe] No, no, no, no- Oscar, you don't have to go off. I remembered throwing it back out, but I missed you.  Ar

In [ ]:
import torch
from tqdm import tqdm

def evaluate_model(text, model):
  max_length = model.config.n_positions
  stride = 512
  
  nlls = []
  for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs[0] * trg_len

    nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)



Using the "Dialogue model", which was based on conversations between Jim and DWIGHT, to create a conversation between the characters  

In [70]:
result_dialogue = predict_multiple_steps('JIM: What are you doing?', one_step_model_dialogue)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [71]:
result_dialogue

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"JIM: What are you doing? I know it's a big mistake. I think I should just get with me and someone started getting off the person some Phyllis and Geese stress do you two mencidency of water?  We'll be out of things. And, I dunno a little boy... with stock. You just got listening here. You're fine, is she? Ed Tradian is past himself into the game and stink schedule for me to sell rape because of her form bringing the grain of lips. It's a durber. I want to be Nicollend, Andy! It's a couple minutes from 10's 50 Kevin return. So well, great relationsh all that-- well, I got a call from vacation Nellow, sweet bigger than Guestion in Rossum. I signed up every single one of the budgets to sit here that you've had one on those nightly lets break that I will return the greased flush] I'm going to pop off the pipes! [accents from the kreeging the mouths. But what people wait, even if share in the paper? Not coming out first. You can't do tha

Using the preprepared GPT2 model to generate Dialogue

In [73]:
from transformers import pipeline
prompt="Oh, I told you. I couldn't close it. So..."
generator = pipeline('text-generation', model = 'gpt2')
output = generator("The following is a dialogue between Jim and Dwight from 'The Office'.\nJim:{}\nDweight: I don't want to!".format(prompt), max_length = 250, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [74]:
prompt="Oh, I told you. I couldn't close it. So..."
output = generator("The following is a dialogue between Jim and Dwight from 'The Office'.\nJim:{}\nDweight:  tI don't wanto!".format(prompt), max_length = 250, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [75]:
for i in range(len(output)):
  print(output[i]["generated_text"])
  print("====================================")

The following is a dialogue between Jim and Dwight from 'The Office'.
Jim:Oh, I told you. I couldn't close it. So...
Dweight:  tI don't wanto!
Jim: Okay...
Dyst: Why not?
Carson: Oh...
Jim: You really mean the other side?
Dyst: Not really.
Jobs: Why would you want a job with...
Dyst...
Jim: Well, one, the one that will be good for you, one that will give you the kind of job you'd want, that you'd want as soon as you get to where you're going, and the other one you just want, that will bring in some extra money in the company or help with the salary, so it's not like I say, 'I want $200 a year. Why don't I give you $730? I want to make more money', [Dyst] says something like...
Dhyst: That's the reason I started talking to it just today--
Jeff: You don't. *laughs*
Jeff: So I think you're right that jobs are really, really important
The following is a dialogue between Jim and Dwight from 'The Office'.
Jim:Oh, I told you. I couldn't close it. So...
Dweight:  tI don't wanto!
Shocked as Dwi

In [76]:
output

[{'generated_text': "The following is a dialogue between Jim and Dwight from 'The Office'.\nJim:Oh, I told you. I couldn't close it. So...\nDweight:  tI don't wanto!\nJim: Okay...\nDyst: Why not?\nCarson: Oh...\nJim: You really mean the other side?\nDyst: Not really.\nJobs: Why would you want a job with...\nDyst...\nJim: Well, one, the one that will be good for you, one that will give you the kind of job you'd want, that you'd want as soon as you get to where you're going, and the other one you just want, that will bring in some extra money in the company or help with the salary, so it's not like I say, 'I want $200 a year. Why don't I give you $730? I want to make more money', [Dyst] says something like...\nDhyst: That's the reason I started talking to it just today--\nJeff: You don't. *laughs*\nJeff: So I think you're right that jobs are really, really important"},
 {'generated_text': 'The following is a dialogue between Jim and Dwight from \'The Office\'.\nJim:Oh, I told you. I coul

## FINETUNE GPT2

In [77]:

import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup


In [78]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [80]:
text = text.replace(":\n", ":")

In [81]:
exampels = []
stop_at = 5

example = ""
counter = 0

for t in text.split("\n"):
  example+=t+"\n"
  counter += 1

  if counter == stop_at:
    exampels.append(example)
    counter = 0 
    example = ""

In [82]:
#import torch
#from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler


class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer(txt, truncation=True, max_length=max_length)

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [83]:
dataset = GPT2Dataset(exampels, tokenizer)

In [84]:
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

10,782 training samples
1,199 validation samples


In [85]:
batch_size = 1
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [86]:
import random

# I'm not really doing anything with the config buheret
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.to(device)

# Set the seed value all over the place to make this reproducible.
seed_val = 42


In [87]:
# some parameters I cooked up that work reasonably well

epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [88]:
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)
import datetime

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [89]:
import time
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        
        
        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of  10,782. Loss: 2.631683826446533.   Elapsed: 0:00:10.
0:  reactorsIM:Hey, my name's, uh, this is not'sister'.
BARTENDER'S:Hey, um.
BARTENDER'S:Yeah, I understand what your...
BARTENDER'S:Ah...
BARTENDER'S:Hey, you''.
PHYLLIS'S:Oh, right, I, man. Ah, I, man.
PHYLLIS'Hey, here's a guy. I got something to thank you.
PHYLLIS'Hey, here's a guy. Oh, man, what about you, you know, here's the guy.
PHYLLIS'Hey, this is a really nice guy. You can be my sweet girlfriend.
PHYLLIS'Hey, it's a nice guy. Hey, you know, I'm super smart.
PHYLLIS'Hey, here's the guy. And I


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   200  of  10,782. Loss: 3.8089983463287354.   Elapsed: 0:00:21.
0:  VladOBY:No, no. No, it's not.
DAVID:So what is the size of this animal? I can only ask you.
ERIN:Cactus? [pause] Well, it is not a big guy and very not a big thing. But it is a tiny animal... when it's a tiny animal you should not worry too much about that. If we get the animal in your mouth, you can give it a bite. And if you put the animal back in your mouth, you can eat it.
JIM:It's still cute.
PAM:Big?
JIM:Yeah, it's cute. It is cute.
KARRY:Dwight, you'll like a bite.
JIM:Yes, you do. I will. I'm going out for a while. I'll wait. And I'll just try and find out.
ROAN:Okay, I have a crazy little


...

  Batch 10,500  of  10,782. Loss: 2.603227376937866.   Elapsed: 0:18:58.
0: olutionKE:Jim...
JIM:You gotta be kidding me.
RYAN:Hey, what's going on?
JIM:Um, I don't know, is it true that there are a bunch of people over here that could get us kicked off the bus?
RYAN:Um, what if the bus drives away? 'Cause it's the same as the bus.
JIM:Oh, man! Right. Okay. Okay.
RYAN:How long could we keep that in a n slip? [walking away]
PAM:Oh, uh, six months.
RYAN:Yeah, three months?
JIM:Ah, six months.
RYAN:Oh, I guess I'm still dating Pam.
PAM:Do you have any idea what I've been through so much lately?
RYAN:Um, well, you know what, when you actually start dating people, you know, it's not


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 10,600  of  10,782. Loss: 0.9071469902992249.   Elapsed: 0:19:09.
0: alingTERN:Hi.
PAM:Hey, thanks.
CORPORATE:Yeah, I'll be with you in a second.
PAM:Okay.
CORPORATE:You and I are just close. We are in Pennsylvania. The Perfectinschlag is a tribute to the team. It is Pam's mom.
PAM:Yes.
CORPORATE:Pam, mom is our daddy now. Mom is the cool, funny, funny, funny guy around here. Mom is kind of a nice guy. He's like, everything alright? Except for Ryan and Holly. But I would be like, 'You're perfect, let's see what you're doing.'
PAM:Oh... yeah. Um... Hey. What's up?
CORPORATE:What's up? Come on. How about this? How about we put you in charge? You said your mom would be the most incredible guy in the world, and they


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 10,700  of  10,782. Loss: 2.3571574687957764.   Elapsed: 0:19:20.
0:  pipesENATOR LIPTON:No it's not.
MICHAEL:No it is not.
DWIGHT:Please. [starts booing everyone]
ERIN:So this isn't matching up with our customer, but there is a guy that's making a big deal out of this. [camera moves around to reveal that he is related to Michael] He is never to me.
KELLY:He is
OSCAR:That's so false.
KELLY:If he was driving, then maybe he would say he was driving a green card.
OSCAR:What?
KELLY:If it's driving them crazy...
ERIN:He says'green card'] It goes in green card.
OSCAR:He said green card.
KEVIN:Huh? What's that, like, on merit? Did you even know that Kevin is a green card? And if I knew that, I wouldn't know

  Average training loss: 1.38
  Training epoch took: 0:19:29

Running Validation...
  Validation Loss: 3.75
  Validation took: 0:00:19

Training complete!
Total training took 1:38:37 (h:mm:ss)


## generate from finetuned model

In [90]:
torch.save(model.state_dict(), "model.pt")

In [91]:
#from transformers import pipeline
prompt="Oh, I told you. I couldn't close it. So..."
generator = pipeline('text-generation', model = model.cpu(), tokenizer=tokenizer)
output = generator("The following is a dialogue between Jim and Dwight from 'The Office'.\nJim:{}\nDweight: I don't want to!".format(prompt), max_length = 250, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [92]:
for i in range(len(output)):
  print(output[i]["generated_text"])
  print("====================================")

The following is a dialogue between Jim and Dwight from 'The Office'.
Jim:Oh, I told you. I couldn't close it. So...
Dweight: I don't want to! It's for my suit!
JIM:You know, Dwight comes from the office and comes from the office and says the same twelve clichmouth.
DWIGHT:Yes! Ha. Oh! Yeah! Ha ha ha-ha!
DWIGHT:Dwight Schrute can be your father. He was my father and he worked part-time frozen. He ate the refrigerator for years and his family never ate his life. He ate the refrigerator for no good reason.
DWIGHT:My mom and I are saving the refrigerator for the wedding. My mother's giving me a gift to wait till the wedding, so, hold on, hold on hold, hold on for a second. Hold on one second. [to Jim] I'm so sorry. I just want you to be able to avoid all these awkward situations and get back on your own.
ANDY:This is the perfect ice cream ice cream conversations. [throws cake at Kevin and hands it to Dwight]
DWIGHT:This is a
The following is a dialogue between Jim and Dwight from 'The Off